In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features_names = [
    'cement', 'water', 'superplasticizer', 'age'   
]
target_name = 'concrete_compressive_strength'
# Load the data
data = pd.read_excel('concrete_data.xlsx')

print(data.head())
# Print the headers (column names)
print("Headers:", data.columns.tolist())

print("Data shape:", data.shape)


   cement  water  superplasticizer  age  concrete_compressive_strength
0   540.0  162.0               2.5   28                          79.99
1   540.0  162.0               2.5   28                          61.89
2   332.5  228.0               0.0  270                          40.27
3   332.5  228.0               0.0  365                          41.05
4   198.6  192.0               0.0  360                          44.30
Headers: ['cement', 'water', 'superplasticizer', 'age', 'concrete_compressive_strength']
Data shape: (700, 5)


In [ ]:
x = data[features_names]
y = data[target_name]

scaller = StandardScaler()

def normalize(data,columns, is_test=False):
    if is_test:
        data = scaller.transform(data)
    else:
        data = scaller.fit_transform(data)
    return pd.DataFrame(data, columns=columns)
      
   
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, shuffle=True)
x_train = normalize(x_train, features_names)
x_test = normalize(x_test, is_test=True, columns=features_names)
y_train = normalize(y_train.values.reshape(-1, 1), columns=[target_name])
y_test = normalize(y_test.values.reshape(-1, 1), is_test=True, columns=[target_name])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(525, 4) (175, 4) (525, 1) (175, 1)


count    700.000000
mean      37.642514
std       17.607925
min        2.330000
25%       24.280000
50%       36.615000
75%       50.120000
max       82.600000
Name: concrete_compressive_strength, dtype: float64

In [29]:

class NeuralNetwork:
    def __init__(self,input_size, hidden_size, output_size,epochs=100, learning_rate=0.01):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.init_weights()
        self.initialize_bais()
        
        
    def init_weights(self):
        self.weights1 = np.random.randn(self.input_size, self.hidden_size)
        self.weights2 = np.random.randn(self.hidden_size, self.output_size)
        self.bias1 = np.random.randn(1, self.hidden_size)
        
    def initialize_bais(self):
        self.hidden_layer_bais = np.random.randn((1, self.hidden_size))
        self.output_layer_bais = np.random.randn((1, self.output_size))
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, x):
        hidden_layer_input = np.dot(x, self.weights1) + self.hidden_layer_bais
        hidden_layer_output = self.sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, self.weights2) + self.output_layer_bais
        output_layer_output = self.sigmoid(output_layer_input)
        return output_layer_output
    
    def backward(self, x, y, output):
        pass #TODO
    
    def train(self, x, y):
        self.x = x
        self.y = y
        for i in range(self.epochs):
            output = self.forward(x)
            self.backward(x, y, output)
    
    def predict(self, x):
        return self.forward(x)
        
        
        
    
        
        